## Importing Libraries

In [1]:
import numpy as numpy
import pandas as pd
import difflib #to get the closet match of some value
from sklearn.feature_extraction.text import TfidfVectorizer #convert textual data into numerical data(feature vector)
from sklearn.metrics.pairwise import  cosine_similarity 

## Importing Dataset

In [2]:
movies_data=pd.read_csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")
movie_keywords = pd.read_csv("../input/the-movies-dataset/keywords.csv")
movie_credits = pd.read_csv("../input/the-movies-dataset/credits.csv")


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
movies_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
movie_keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [5]:
movie_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [6]:
movies_data = movies_data[movies_data['vote_count']>=55]

In [7]:
movie_credits = movie_credits[['id','cast']]

In [8]:
# Removing the records for which the id is not available
movies_data = movies_data[movies_data['id'].str.isnumeric()]

In [9]:
movies_data['id'] = movies_data['id'].astype(int)
df = pd.merge(movies_data, movie_keywords, on='id', how='left')
df.reset_index(inplace=True, drop=True)
df = pd.merge(df, movie_credits, on='id', how='left')
df.reset_index(inplace=True, drop=True)

In [10]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords,cast
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c..."
3,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '..."
4,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '...","[{'cast_id': 25, 'character': 'Lt. Vincent Han..."


In [11]:
df.shape

(8770, 26)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8770 entries, 0 to 8769
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  8770 non-null   object 
 1   belongs_to_collection  1955 non-null   object 
 2   budget                 8770 non-null   object 
 3   genres                 8770 non-null   object 
 4   homepage               2727 non-null   object 
 5   id                     8770 non-null   int64  
 6   imdb_id                8770 non-null   object 
 7   original_language      8770 non-null   object 
 8   original_title         8770 non-null   object 
 9   overview               8735 non-null   object 
 10  popularity             8770 non-null   object 
 11  poster_path            8770 non-null   object 
 12  production_companies   8770 non-null   object 
 13  production_countries   8770 non-null   object 
 14  release_date           8769 non-null   object 
 15  reve

In [13]:
df.isnull().sum()

adult                       0
belongs_to_collection    6815
budget                      0
genres                      0
homepage                 6043
id                          0
imdb_id                     0
original_language           0
original_title              0
overview                   35
popularity                  0
poster_path                 0
production_companies        0
production_countries        0
release_date                1
revenue                     0
runtime                     1
spoken_languages            0
status                      1
tagline                  1883
title                       0
video                       0
vote_average                0
vote_count                  0
keywords                    0
cast                        0
dtype: int64

In [14]:
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in eval(x)])
df['genres'] = df['genres'].apply(lambda x: ' '.join([i.replace(" ","") for i in x]))
df['keywords'] = df['keywords'].apply(lambda x: [i['name'] for i in eval(x)])
df['keywords'] = df['keywords'].apply(lambda x: ' '.join([i.replace(" ",'') for i in x]))
df['cast'] = df['cast'].apply(lambda x: [i['name'] for i in eval(x)])
df['cast'] = df['cast'].apply(lambda x: ' '.join([i.replace(" ",'') for i in x]))

In [15]:
selected_features=['genres','title','tagline','keywords','cast','production_companies']

In [16]:
for feature in selected_features:
    df[feature]=df[feature].fillna('')

In [17]:
cmf=df['genres']+' '+df['title']+' '+df['tagline']+' '+df['keywords']+' '+df['cast']+' '+df['production_companies']

In [18]:
cmf

0       Animation Comedy Family Toy Story  jealousy to...
1       Adventure Fantasy Family Jumanji Roll the dice...
2       Romance Comedy Grumpier Old Men Still Yelling....
3       Comedy Father of the Bride Part II Just When H...
4       Action Crime Drama Thriller Heat A Los Angeles...
                              ...                        
8765    Comedy With Open Arms Thanks for the invitatio...
8766    Comedy The Visitors: Bastille Day  nazis castl...
8767    Action Adventure Thriller Titanic 2 100 years ...
8768    Family Animation Romance Comedy In a Heartbeat...
8769    Comedy Cadet Kelly Too Cool For The Rules! mil...
Length: 8770, dtype: object

In [19]:
# converting text data into feature vectors
vec=TfidfVectorizer(max_features=5000)

In [20]:
feature_vec=vec.fit_transform(cmf)

In [21]:
print(feature_vec)

  (0, 2213)	0.03496716395552374
  (0, 4384)	0.1393798200513308
  (0, 3663)	0.21028258722881576
  (0, 3342)	0.03496716395552374
  (0, 2808)	0.23827558043103225
  (0, 2793)	0.24511785385613805
  (0, 2533)	0.22549964906860906
  (0, 4821)	0.21875438004329986
  (0, 1465)	0.25368692783085006
  (0, 4552)	0.23827558043103225
  (0, 4590)	0.20829579089740122
  (0, 4628)	0.24706678563781934
  (0, 3918)	0.20894527764953538
  (0, 1842)	0.17849856662339691
  (0, 1843)	0.1611884949873595
  (0, 798)	0.188758098599538
  (0, 2400)	0.18986404496011977
  (0, 4367)	0.14512239477880506
  (0, 4627)	0.4735198152733421
  (0, 1690)	0.10369928245485745
  (0, 1148)	0.06912244167230275
  (0, 481)	0.24247374048126044
  (1, 8)	0.2589342751897002
  (1, 1157)	0.21262138305977846
  (1, 2275)	0.2589342751897002
  :	:
  (8768, 454)	0.13474102603413757
  (8768, 4513)	0.138112445042015
  (8768, 2213)	0.042354273249964804
  (8768, 3342)	0.042354273249964804
  (8768, 1690)	0.12560663342628609
  (8768, 1148)	0.083725142422681

In [22]:
similarity=cosine_similarity(feature_vec)

In [23]:
print(similarity)

[[1.         0.01939365 0.00866011 ... 0.00345618 0.05738169 0.01471848]
 [0.01939365 1.         0.01408909 ... 0.03513104 0.05644298 0.03713516]
 [0.00866011 0.01408909 1.         ... 0.00619127 0.03782561 0.03900451]
 ...
 [0.00345618 0.03513104 0.00619127 ... 1.         0.01531501 0.01817906]
 [0.05738169 0.05644298 0.03782561 ... 0.01531501 1.         0.02739475]
 [0.01471848 0.03713516 0.03900451 ... 0.01817906 0.02739475 1.        ]]


In [24]:
similarity.shape

(8770, 8770)

In [25]:
def recommendation(movie_title):
    id_of_movie = df[df['title']==movie_title].index[0]
    similarity_score = similarity[id_of_movie]
    movie_list = sorted(list(enumerate(similarity_score)), reverse=True, key=lambda x:x[1])[1:20]
    for i in movie_list:
        print(df.iloc[i[0]].title)

In [26]:
recommendation('Death Note')

Death Note: The Last Name
L: Change the World
Battle of the Bulge
Mirrormask
Scooby-Doo! The Mystery Begins
Rurouni Kenshin
Battle Royale
Berserk: The Golden Age Arc 2 - The Battle for Doldrey
Deathtrap
Running on Empty
Confessions
10,000 BC
Gossip
The Bad Seed
On the Other Side of the Tracks
Terminus
Terminus
Head Full of Honey
Rurouni Kenshin: Kyoto Inferno
